In [116]:
import os
import json
import pandas as pd

folder_path = '/home/jupyter-23521027/refresh-bert/data/liputan6_data/canonical/test' 

file_list = os.listdir(folder_path)

In [117]:
json_data_list = []

file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)  
    
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            
            json_data_list.append(json_data)

df = pd.DataFrame(json_data_list)

In [118]:
df

,id,url,clean_article,clean_summary,extractive_summary
0,19962,https://www.liputan6.com/news/read/19962/gas-b...,"[[Liputan6, ., com, ,, Jakarta, :, Untuk, meme...","[[Pertamina, akan, menyalurkan, gas, alam, dar...","[0, 4]"
1,23319,https://www.liputan6.com/news/read/23319/pelaj...,"[[Liputan6, ., com, ,, Yogyakarta, :, Sekitar,...","[[Ratusan, siswa, Sekolah, Menangah, Umum, 6, ...","[1, 2]"
2,18513,https://www.liputan6.com/news/read/18513/bamba...,"[[Liputan6, ., com, ,, Jakarta, :, Ekspektasi,...","[[Tim, Ekonomi, Pemerintah, yang, sedang, beru...","[0, 6]"
3,23951,https://www.liputan6.com/news/read/23951/bengk...,"[[Liputan6, ., com, ,, Jakarta, :, Perampokan,...","[[Bengkel, Toyota, Astra, dan, Indomaret, di, ...","[0, 1]"
4,13167,https://www.liputan6.com/news/read/13167/bentr...,"[[Liputan6, ., com, ,, Jakarta, :, Bentrokan, ...","[[Bentrokan, antara, mahasiswa, satu, kampus, ...","[0, 7]"
...,...,...,...,...,...
10967,15906,https://www.liputan6.com/news/read/15906/soepa...,"[[Liputan6, ., com, ,, Jakarta, :, Ketua, DPR,...","[[Kepergian, Baharuddin, Lopa, membuat, Akbar,...","[0, 3]"
10968,19241,https://www.liputan6.com/news/read/19241/golka...,"[[Liputan6, ., com, ,, Jakarta, :, Partai, Gol...","[[Ketua, Umum, DPP, Partai, Golkar, Akbar, Tan...","[2, 3]"
10969,20408,https://www.liputan6.com/news/read/20408/pemer...,"[[Liputan6, ., com, ,, Jakarta, :, Pemerintah,...","[[Pemerintah, akan, memberlakukan, sistem, UMR...","[1, 4, 5]"
10970,26145,https://www.liputan6.com/news/read/26145/pemer...,"[[Liputan6, ., com, ,, Medan, :, Pemerintah, t...","[[Biro, perjalanan, haji, yang, menggunakan, p...","[1, 5]"


In [119]:
df['sentences'] = df['clean_article'].apply(lambda x: [' '.join(sentence) for sentence in x])

In [120]:
df

,id,url,clean_article,clean_summary,extractive_summary,sentences
0,19962,https://www.liputan6.com/news/read/19962/gas-b...,"[[Liputan6, ., com, ,, Jakarta, :, Untuk, meme...","[[Pertamina, akan, menyalurkan, gas, alam, dar...","[0, 4]","[Liputan6 . com , Jakarta : Untuk memenuhi ken..."
1,23319,https://www.liputan6.com/news/read/23319/pelaj...,"[[Liputan6, ., com, ,, Yogyakarta, :, Sekitar,...","[[Ratusan, siswa, Sekolah, Menangah, Umum, 6, ...","[1, 2]","[Liputan6 . com , Yogyakarta : Sekitar 700 sis..."
2,18513,https://www.liputan6.com/news/read/18513/bamba...,"[[Liputan6, ., com, ,, Jakarta, :, Ekspektasi,...","[[Tim, Ekonomi, Pemerintah, yang, sedang, beru...","[0, 6]","[Liputan6 . com , Jakarta : Ekspektasi pasar y..."
3,23951,https://www.liputan6.com/news/read/23951/bengk...,"[[Liputan6, ., com, ,, Jakarta, :, Perampokan,...","[[Bengkel, Toyota, Astra, dan, Indomaret, di, ...","[0, 1]","[Liputan6 . com , Jakarta : Perampokan bersenj..."
4,13167,https://www.liputan6.com/news/read/13167/bentr...,"[[Liputan6, ., com, ,, Jakarta, :, Bentrokan, ...","[[Bentrokan, antara, mahasiswa, satu, kampus, ...","[0, 7]","[Liputan6 . com , Jakarta : Bentrokan antara m..."
...,...,...,...,...,...,...
10967,15906,https://www.liputan6.com/news/read/15906/soepa...,"[[Liputan6, ., com, ,, Jakarta, :, Ketua, DPR,...","[[Kepergian, Baharuddin, Lopa, membuat, Akbar,...","[0, 3]","[Liputan6 . com , Jakarta : Ketua DPR Akbar Ta..."
10968,19241,https://www.liputan6.com/news/read/19241/golka...,"[[Liputan6, ., com, ,, Jakarta, :, Partai, Gol...","[[Ketua, Umum, DPP, Partai, Golkar, Akbar, Tan...","[2, 3]","[Liputan6 . com , Jakarta : Partai Golongan Ka..."
10969,20408,https://www.liputan6.com/news/read/20408/pemer...,"[[Liputan6, ., com, ,, Jakarta, :, Pemerintah,...","[[Pemerintah, akan, memberlakukan, sistem, UMR...","[1, 4, 5]","[Liputan6 . com , Jakarta : Pemerintah berenca..."
10970,26145,https://www.liputan6.com/news/read/26145/pemer...,"[[Liputan6, ., com, ,, Medan, :, Pemerintah, t...","[[Biro, perjalanan, haji, yang, menggunakan, p...","[1, 5]","[Liputan6 . com , Medan : Pemerintah tetap tak..."


In [121]:
df.drop('clean_article', axis=1, inplace=True)

In [122]:
df

,id,url,clean_summary,extractive_summary,sentences
0,19962,https://www.liputan6.com/news/read/19962/gas-b...,"[[Pertamina, akan, menyalurkan, gas, alam, dar...","[0, 4]","[Liputan6 . com , Jakarta : Untuk memenuhi ken..."
1,23319,https://www.liputan6.com/news/read/23319/pelaj...,"[[Ratusan, siswa, Sekolah, Menangah, Umum, 6, ...","[1, 2]","[Liputan6 . com , Yogyakarta : Sekitar 700 sis..."
2,18513,https://www.liputan6.com/news/read/18513/bamba...,"[[Tim, Ekonomi, Pemerintah, yang, sedang, beru...","[0, 6]","[Liputan6 . com , Jakarta : Ekspektasi pasar y..."
3,23951,https://www.liputan6.com/news/read/23951/bengk...,"[[Bengkel, Toyota, Astra, dan, Indomaret, di, ...","[0, 1]","[Liputan6 . com , Jakarta : Perampokan bersenj..."
4,13167,https://www.liputan6.com/news/read/13167/bentr...,"[[Bentrokan, antara, mahasiswa, satu, kampus, ...","[0, 7]","[Liputan6 . com , Jakarta : Bentrokan antara m..."
...,...,...,...,...,...
10967,15906,https://www.liputan6.com/news/read/15906/soepa...,"[[Kepergian, Baharuddin, Lopa, membuat, Akbar,...","[0, 3]","[Liputan6 . com , Jakarta : Ketua DPR Akbar Ta..."
10968,19241,https://www.liputan6.com/news/read/19241/golka...,"[[Ketua, Umum, DPP, Partai, Golkar, Akbar, Tan...","[2, 3]","[Liputan6 . com , Jakarta : Partai Golongan Ka..."
10969,20408,https://www.liputan6.com/news/read/20408/pemer...,"[[Pemerintah, akan, memberlakukan, sistem, UMR...","[1, 4, 5]","[Liputan6 . com , Jakarta : Pemerintah berenca..."
10970,26145,https://www.liputan6.com/news/read/26145/pemer...,"[[Biro, perjalanan, haji, yang, menggunakan, p...","[1, 5]","[Liputan6 . com , Medan : Pemerintah tetap tak..."


In [123]:
df['summary_sentences'] = df['clean_summary'].apply(lambda x: [' '.join(sentence) for sentence in x])

In [124]:
df

,id,url,clean_summary,extractive_summary,sentences,summary_sentences
0,19962,https://www.liputan6.com/news/read/19962/gas-b...,"[[Pertamina, akan, menyalurkan, gas, alam, dar...","[0, 4]","[Liputan6 . com , Jakarta : Untuk memenuhi ken...",[Pertamina akan menyalurkan gas alam dari Suma...
1,23319,https://www.liputan6.com/news/read/23319/pelaj...,"[[Ratusan, siswa, Sekolah, Menangah, Umum, 6, ...","[1, 2]","[Liputan6 . com , Yogyakarta : Sekitar 700 sis...",[Ratusan siswa Sekolah Menangah Umum 6 Yogyaka...
2,18513,https://www.liputan6.com/news/read/18513/bamba...,"[[Tim, Ekonomi, Pemerintah, yang, sedang, beru...","[0, 6]","[Liputan6 . com , Jakarta : Ekspektasi pasar y...",[Tim Ekonomi Pemerintah yang sedang berunding ...
3,23951,https://www.liputan6.com/news/read/23951/bengk...,"[[Bengkel, Toyota, Astra, dan, Indomaret, di, ...","[0, 1]","[Liputan6 . com , Jakarta : Perampokan bersenj...",[Bengkel Toyota Astra dan Indomaret di Jalan R...
4,13167,https://www.liputan6.com/news/read/13167/bentr...,"[[Bentrokan, antara, mahasiswa, satu, kampus, ...","[0, 7]","[Liputan6 . com , Jakarta : Bentrokan antara m...",[Bentrokan antara mahasiswa satu kampus terjad...
...,...,...,...,...,...,...
10967,15906,https://www.liputan6.com/news/read/15906/soepa...,"[[Kepergian, Baharuddin, Lopa, membuat, Akbar,...","[0, 3]","[Liputan6 . com , Jakarta : Ketua DPR Akbar Ta...",[Kepergian Baharuddin Lopa membuat Akbar Tandj...
10968,19241,https://www.liputan6.com/news/read/19241/golka...,"[[Ketua, Umum, DPP, Partai, Golkar, Akbar, Tan...","[2, 3]","[Liputan6 . com , Jakarta : Partai Golongan Ka...",[Ketua Umum DPP Partai Golkar Akbar Tandjung m...
10969,20408,https://www.liputan6.com/news/read/20408/pemer...,"[[Pemerintah, akan, memberlakukan, sistem, UMR...","[1, 4, 5]","[Liputan6 . com , Jakarta : Pemerintah berenca...",[Pemerintah akan memberlakukan sistem UMR baru...
10970,26145,https://www.liputan6.com/news/read/26145/pemer...,"[[Biro, perjalanan, haji, yang, menggunakan, p...","[1, 5]","[Liputan6 . com , Medan : Pemerintah tetap tak...",[Biro perjalanan haji yang menggunakan paspor ...


In [125]:
df['summary_string'] = df['summary_sentences'].apply(lambda x: ' '.join(x))

In [126]:
df.drop('clean_summary', axis=1, inplace=True)
df.drop('summary_sentences', axis=1, inplace=True)

In [127]:
df

,id,url,extractive_summary,sentences,summary_string
0,19962,https://www.liputan6.com/news/read/19962/gas-b...,"[0, 4]","[Liputan6 . com , Jakarta : Untuk memenuhi ken...",Pertamina akan menyalurkan gas alam dari Sumat...
1,23319,https://www.liputan6.com/news/read/23319/pelaj...,"[1, 2]","[Liputan6 . com , Yogyakarta : Sekitar 700 sis...",Ratusan siswa Sekolah Menangah Umum 6 Yogyakar...
2,18513,https://www.liputan6.com/news/read/18513/bamba...,"[0, 6]","[Liputan6 . com , Jakarta : Ekspektasi pasar y...",Tim Ekonomi Pemerintah yang sedang berunding d...
3,23951,https://www.liputan6.com/news/read/23951/bengk...,"[0, 1]","[Liputan6 . com , Jakarta : Perampokan bersenj...",Bengkel Toyota Astra dan Indomaret di Jalan Ra...
4,13167,https://www.liputan6.com/news/read/13167/bentr...,"[0, 7]","[Liputan6 . com , Jakarta : Bentrokan antara m...",Bentrokan antara mahasiswa satu kampus terjadi...
...,...,...,...,...,...
10967,15906,https://www.liputan6.com/news/read/15906/soepa...,"[0, 3]","[Liputan6 . com , Jakarta : Ketua DPR Akbar Ta...",Kepergian Baharuddin Lopa membuat Akbar Tandju...
10968,19241,https://www.liputan6.com/news/read/19241/golka...,"[2, 3]","[Liputan6 . com , Jakarta : Partai Golongan Ka...",Ketua Umum DPP Partai Golkar Akbar Tandjung me...
10969,20408,https://www.liputan6.com/news/read/20408/pemer...,"[1, 4, 5]","[Liputan6 . com , Jakarta : Pemerintah berenca...",Pemerintah akan memberlakukan sistem UMR baru ...
10970,26145,https://www.liputan6.com/news/read/26145/pemer...,"[1, 5]","[Liputan6 . com , Medan : Pemerintah tetap tak...",Biro perjalanan haji yang menggunakan paspor h...


In [128]:
# https://github.com/google-research/google-research/blob/master/rouge/rouge_scorer.py
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(
    'IDX Incubator \xa0  mengadakan sesi diskusi dengan tema “ Technology vs Humanity ” ',
    'IDX Incubator adakan sesi diskusi dengan tema Technology vs Humanity'
)
print(scores)

{'rouge1': Score(precision=0.9, recall=0.9, fmeasure=0.9), 'rouge2': Score(precision=0.7777777777777778, recall=0.7777777777777778, fmeasure=0.7777777777777778), 'rougeL': Score(precision=0.9, recall=0.9, fmeasure=0.9)}


In [129]:
from datetime import datetime

DATASET_SIZE = 5000
all_document_sentences_f1_rouge_scores = []

now = datetime.now()
df = df.head(DATASET_SIZE)

for i, row in df.iterrows():
    document_sentences_f1_rouge_scores = []
    for sentence in row["sentences"]:
        scores = scorer.score(sentence, row["summary_string"])
        rouge1_f1 = scores["rouge1"].fmeasure
        rouge2_f1 = scores["rouge2"].fmeasure
        rougeL_f1 = scores["rougeL"].fmeasure
        rouge_score = (rouge1_f1 + rouge2_f1 + rougeL_f1) / 3.0
        document_sentences_f1_rouge_scores.append(rouge_score)
        
    if (i + 1) % 1000 == 0:
        print(f"processing {i + 1} with time: {datetime.now() - now}")
        now = datetime.now()
        
    all_document_sentences_f1_rouge_scores.append(document_sentences_f1_rouge_scores)

df["document_sentences_f1_rouge_scores"] = all_document_sentences_f1_rouge_scores
df.tail(5)

processing 1000 with time: 0:00:09.037368
processing 2000 with time: 0:00:08.891598
processing 3000 with time: 0:00:09.104161
processing 4000 with time: 0:00:08.721489
processing 5000 with time: 0:00:09.365658


/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,url,extractive_summary,sentences,summary_string,document_sentences_f1_rouge_scores
4995,18900,https://www.liputan6.com/news/read/18900/lima-...,"[0, 1]","[Liputan6 . com , Jakarta : Penyelundupan senj...",Mengantisipasi penyelundupan senjata dari Seme...,"[0.26608187134502925, 0.49447571398790907, 0.0..."
4996,21731,https://www.liputan6.com/news/read/21731/ppp-m...,"[0, 1]","[Liputan6 . com , Jakarta : Partai Persatuan P...","Mulai hari ini , PPP menggelar Musyawarah Kerj...","[0.27012987012987016, 0.5141739980449658, 0.08..."
4997,19400,https://www.liputan6.com/news/read/19400/karya...,"[0, 3, 5]","[Liputan6 . com , Palembang : Firly , seorang ...",Seorang karyawan diskotik memperdagangkan eskt...,"[0.23910422049956934, 0.0, 0.04444444444444443..."
4998,17737,https://www.liputan6.com/news/read/17737/sejum...,"[0, 1]","[Liputan6 . com , Jakarta : Sebagian anggota F...",Fraksi Kebangkitan Bangsa menggelar rapat kerj...,"[0.43939393939393945, 0.26139817629179335, 0.0..."
4999,23679,https://www.liputan6.com/news/read/23679/kemat...,"[8, 48, 2]","[Liputan6 . com , Jakarta : Pascakematian Ketu...","Menyusul kematian Ketua PDP Theys Hiyo Eluay ,...","[0.2283333333333333, 0.04040404040404041, 0.13..."


In [130]:
all_top_10_sentences = [] 
for i, row in df.iterrows():
    document_sentences_and_scores = [] 
    for sentences_and_scores in zip(
        row["sentences"], 
        row["document_sentences_f1_rouge_scores"], 
        [x+1 for x in range(len(row["sentences"]))]
    ):
        document_sentences_and_scores.append(sentences_and_scores)

    top_10_sentences = sorted(document_sentences_and_scores, key=lambda x: x[1], reverse=True)[:10]
    all_top_10_sentences.append([(x[0],x[2]) for x in top_10_sentences])

df["top_10_sentences"] = all_top_10_sentences
df.head(1)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,url,extractive_summary,sentences,summary_string,document_sentences_f1_rouge_scores,top_10_sentences
0,19962,https://www.liputan6.com/news/read/19962/gas-b...,"[0, 4]","[Liputan6 . com , Jakarta : Untuk memenuhi ken...",Pertamina akan menyalurkan gas alam dari Sumat...,"[0.30026455026455023, 0.05925925925925926, 0.1...",[(Proyek transmisi gas ini untuk mendukung keb...


In [131]:
import itertools

df_candidate_summary = pd.DataFrame()

all_candidate_summary_txt = []
all_candidate_summary_ids_txt = []
all_document_length = [] 
for i, row in df.iterrows():
    document_candidate_summaries = []
    document_candidate_summaries_idx = []

    for itemcount in [1,2,3]:
        candidate_summaries = list(itertools.combinations(list(row["top_10_sentences"]), itemcount))
        for candidate_summary in candidate_summaries:
            candidate_summary_txt = " ".join([val[0] for val in candidate_summary])
            candidate_summary_ids = " ".join([str(val[1]) for val in candidate_summary])

            document_candidate_summaries.append(candidate_summary_txt)
            document_candidate_summaries_idx.append(candidate_summary_ids)

    all_candidate_summary_txt.append(document_candidate_summaries)
    all_candidate_summary_ids_txt.append(document_candidate_summaries_idx)
    all_document_length.append(len(row["sentences"]))

df_candidate_summary["candidate_summaries"] = all_candidate_summary_txt
df_candidate_summary["candidate_summary_ids"] = all_candidate_summary_ids_txt
df_candidate_summary["document_length"] = all_document_length
df_candidate_summary["summary"] = df["summary_string"]
df_candidate_summary.head(1)

,candidate_summaries,candidate_summary_ids,document_length,summary
0,[Proyek transmisi gas ini untuk mendukung kebi...,"[5, 1, 7, 9, 8, 3, 10, 6, 4, 2, 5 1, 5 7, 5 9,...",10,Pertamina akan menyalurkan gas alam dari Sumat...


In [132]:
# input: list of kalimat dalam dokumen dan summary
# output: list of scores 

documents = list(
    zip(
        df_candidate_summary["candidate_summaries"], 
        df_candidate_summary["summary"]
    )
)

def cal_rouge(document):
    candidate_summaries = document[0]
    summary = document[1]
    candidate_summary_scores = []
    for candidate_summary in candidate_summaries:
        scores = scorer.score(candidate_summary, summary)
        rouge1_f1 = scores["rouge1"].fmeasure
        rouge2_f1 = scores["rouge2"].fmeasure
        rougeL_f1 = scores["rougeL"].fmeasure
        rouge_score = (rouge1_f1 + rouge2_f1 + rougeL_f1) / 3.0
        candidate_summary_scores.append(rouge_score)
        
    return candidate_summary_scores


In [133]:
import multiprocess
from multiprocess import Pool
import tqdm

cores = 4
print(f"multiprocessing with {cores} cores")


with Pool(processes=cores) as pool:
    results = list(tqdm.tqdm(pool.imap_unordered(cal_rouge, documents), total=len(documents)))

df_candidate_summary["candidate_summary_scores"] = results
df_candidate_summary.tail(5)

multiprocessing with 4 cores


100%|██████████| 5000/5000 [04:43<00:00, 17.66it/s]


,candidate_summaries,candidate_summary_ids,document_length,summary,candidate_summary_scores
4995,[Untuk menghentikan aksi tersebut TNI Angkatan...,"[2, 1, 5, 4, 6, 7, 8, 3, 2 1, 2 5, 2 4, 2 6, 2...",8,Mengantisipasi penyelundupan senjata dari Seme...,"[0.4231025743630785, 0.23014383989993745, 0.09..."
4996,"[Sedianya , Mukernas tersebut bakal membahas p...","[2, 1, 6, 4, 3, 5, 2 1, 2 6, 2 4, 2 3, 2 5, 1 ...",6,"Mulai hari ini , PPP menggelar Musyawarah Kerj...","[0.3474243474243474, 0.33888888888888885, 0.21..."
4997,"[Liputan6 . com , Palembang : Firly , seorang ...","[1, 6, 7, 4, 8, 3, 5, 2, 1 6, 1 7, 1 4, 1 8, 1...",8,Seorang karyawan diskotik memperdagangkan eskt...,"[0.23910422049956934, 0.21805006587615283, 0.1..."
4998,"[Liputan6 . com , Jakarta : Sebagian anggota F...","[1, 2, 4, 7, 5, 3, 6, 8, 1 2, 1 4, 1 7, 1 5, 1...",8,Fraksi Kebangkitan Bangsa menggelar rapat kerj...,"[0.43939393939393945, 0.26139817629179335, 0.1..."
4999,"[Liputan6 . com , Jakarta : Pascakematian Ketu...","[1, 17, 3, 6, 43, 49, 16, 12, 9, 36, 1 17, 1 3...",50,"Menyusul kematian Ketua PDP Theys Hiyo Eluay ,...","[0.2283333333333333, 0.13663337675495732, 0.13..."


In [134]:
all_top_10_candidates = [] 
for i, row in df_candidate_summary.iterrows():
    document_candidates_and_scores = [] 
    for candidates_and_scores in zip(
        row["candidate_summary_ids"], 
        row["candidate_summary_scores"], 
    ):
        document_candidates_and_scores.append(candidates_and_scores)

    top_10_candidates = sorted(document_candidates_and_scores, key=lambda x: x[1], reverse=True)[:10]
    all_top_10_candidates.append(top_10_candidates)

df_candidate_summary["top_10_candidates"] = all_top_10_candidates
df_candidate_summary.head(2)

,candidate_summaries,candidate_summary_ids,document_length,summary,candidate_summary_scores,top_10_candidates
0,[Proyek transmisi gas ini untuk mendukung kebi...,"[5, 1, 7, 9, 8, 3, 10, 6, 4, 2, 5 1, 5 7, 5 9,...",10,Pertamina akan menyalurkan gas alam dari Sumat...,"[0.6027667984189723, 0.21929824561403508, 0.10...","[(5, 0.6027667984189723), (3, 0.55745967741935..."
1,"[Liputan6 . com , Yogyakarta : Sekitar 700 sis...","[1, 2, 3, 8, 7, 5, 9, 6, 10, 4, 1 2, 1 3, 1 8,...",10,Ratusan siswa Sekolah Menangah Umum 6 Yogyakar...,"[0.29337029337029336, 0.22331154684095858, 0.1...","[(1 2, 0.3430308293530178), (1 8, 0.3169318041..."


In [135]:
df_candidate_summary["id"] = df["id"]

with open(f"/home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.test.label.multipleoracle", "w") as file:
    for i, row in df_candidate_summary.iterrows():
        file.write(f"liputan6-{row['id']}\n")
        file.write(f"{row['document_length']}\n")
        for candidate_summary_tuple in row["top_10_candidates"]:
            file.write(f"{candidate_summary_tuple[0]} {str(candidate_summary_tuple[1])[:14]}\n")
        file.write("\n")

In [136]:
!cat /home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.test.label.multipleoracle | head -c 1000

liputan6-19962
10
5 0.602766798418
3 0.557459677419
6 0.468430964416
5 6 0.462568542568
4 0.445519713261
5 4 0.445085470085
10 0.419208306294
5 10 0.409015100292
1 9 0.367610350076
5 2 0.352235772357

liputan6-23319
10
1 2 0.343030829353
1 8 0.316931804168
5 9 0.298031052129
1 0.293370293370
1 10 0.291831357048
6 4 0.287449041547
6 10 0.282212127766
5 10 0.280137772675
1 2 8 0.279693486590
1 7 10 0.277591036414

liputan6-18513
9
2 1 0.383004926108
2 1 9 0.352451641925
2 5 9 0.336174242424
2 5 8 0.331087977138
2 5 6 0.326203208556
2 1 3 0.321949964627
2 1 4 0.318160265528
2 5 0.317970066408
2 4 0.315819944999
2 0.311388311388

liputan6-23951
5
1 0.383116883116
2 5 0.381226053639
1 2 5 0.343609784339
1 3 5 0.336410256410
2 4 0.331113662456
1 2 4 0.323589743589
3 5 0.318341307814
1 2 3 0.312807881773
5 4 0.312169312169
1 3 4 0.298378003296

liputan6-13167
11
8 0.582099506469
8 1 0.522048364153
8 2 0.518847828100
8 6 0.499641577060
8 3 0.483534946236
8 1 2 0.476559616094
8 1 6 0.4710389903